# Import Modules

In [2]:
import json
import pickle

import numpy as np
import pandas as pd
import torch

import utils

In [ ]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Set Vegetable

In [3]:
vegetable = utils.VEGETABLES[0]

json_open = open('./free_params.json', 'r')
params = json.load(json_open)
param = params[vegetable]

print(f"vegetable: {vegetable}")
print(f"free params: {param}")

vegetable: だいこん
free params: {'T': 36, 'batch_size': 2412, 'num_epochs': 64, 'learning_rate': 0.005223420083374704, 'weight_decay': 3.416512393334067e-13, 'eps': 9.671061702182804e-15, 'hidden_size': 368, 'dropout_ratio': 0.5208164295982842, 'is_attention': 1, 'num_ensamble': 5}


# Load Data

In [4]:
train_test = pd.read_csv("./data/mapped_train_test.csv")
train_test["date"] = pd.to_datetime(train_test["date"], format="%Y-%m-%d")
weather = pd.read_csv("./data/sorted_mapped_adjusted_weather.csv")
train_test = pd.concat([train_test, weather], axis=1)
train_test = train_test.reset_index(drop=True)

train_test["year"] = train_test.date.dt.year
years = pd.get_dummies(train_test["year"])
train_test = train_test.drop(columns="year")
train_test = pd.concat([train_test, years], axis=1)

train_test["month"] = train_test.date.dt.month
months = pd.get_dummies(train_test["month"])
train_test = train_test.drop(columns="month")
train_test = pd.concat([train_test, months], axis=1)

areas = pd.get_dummies(train_test["area"])
train_test = train_test.drop(columns="area")
train_test = pd.concat([train_test, areas], axis=1)

test_size = pd.read_csv("./data/test.csv").shape[0]
train = train_test[:-test_size]
test = train_test[-test_size:]

train = utils.get_target_values(train, vegetable)
test = utils.get_target_values(test, vegetable)

changed_col = [1, 0] + [i for i in np.arange(2, train.shape[1])]
train = train[:, changed_col]
test = test[:, changed_col]

test_amount = pd.read_csv("./data/test_amount.csv")
test_amount = test_amount[vegetable].values
test[:, 1] = test_amount

train_loader, train, test, ss = utils.preprocess_data_submit(train, test, T=param["T"], batch_size=param["batch_size"])

# Assign Free Params

In [5]:
future = test.shape[0]
num_ensamble = param["num_ensamble"]
num_epochs = param["num_epochs"]
learning_rate = param["learning_rate"]
weight_decay = param["weight_decay"]
eps = param["eps"]
hidden_size = param["hidden_size"]
dropout_ratio = param["dropout_ratio"]
is_attention = param["is_attention"]

# Training

In [ ]:
models = []
outs = []
hidden_memory_list = []

for _ in range(num_ensamble):
    
    model, out, hidden_memory = utils.pipeline_rnn_submit(train_loader, train, test, future=future, num_epochs=num_epochs, lr=learning_rate,
                                                          weight_decay=weight_decay, eps=eps, hidden_size=hidden_size, dropout_ratio=dropout_ratio, is_attention=is_attention)
    models.append(model)
    outs.append(out)
    hidden_memory_list.append(hidden_memory)

# Predict

In [6]:
preds = []

for model in models:
    with torch.no_grad():
        model.eval()
        pred_y = model.predict(train, test, future)

    pred_y = pred_y.cpu().detach().numpy()
    pred_y = np.concatenate([pred_y.reshape(-1, 1), test[:, 1:]], axis=1)
    pred_y[:, :7] = ss.inverse_transform(pred_y[:, :7])
    pred_y = pred_y[:, 0].tolist()

    preds.append(pred_y)

pred_y = np.mean(np.array(preds),axis=0)

In [7]:
pred_y.tolist()

[969.2453109990744,
 1047.7871674926366,
 1091.5069783712736,
 1134.0139677575103,
 1140.3271676426634,
 1147.0156310811303,
 1148.9660865812402,
 1160.1336844119214,
 1147.7265981596693,
 1155.3268723014764,
 1156.377687523295,
 1159.7946757294173,
 1161.6792124843305,
 1162.132876054244,
 1161.2098309773048,
 1159.3249640579024,
 1156.9438388371095,
 1165.0607044954907,
 1149.9082478308571,
 1138.8714671418027]

# Save state_dict

In [8]:
for idx, model in enumerate(models):
    torch.save(model.state_dict(), f"./models/{vegetable}/rnn_{idx}.pth")

    with open(f"./models/{vegetable}/rnn_{idx}_out.bin", 'wb') as p:
        pickle.dump(outs[idx], p)
    with open(f"./models/{vegetable}/rnn_{idx}_hidden_memory.bin", 'wb') as p:
        pickle.dump(hidden_memory_list[idx], p)    

# Load state_dict

In [6]:
models = []


for idx in range(num_ensamble):
    model = utils.RNN(input_size=train.shape[2], hidden_size=hidden_size,
                      dropout_ratio=dropout_ratio, is_attention=is_attention).to(utils.DEVICE)
    
    model.load_state_dict(torch.load(f"./models/{vegetable}/rnn_{idx}.pth"))
    with open(f"./models/{vegetable}/rnn_{idx}_out.bin", 'rb') as p:
        model.out = pickle.load(p)
    with open(f"./models/{vegetable}/rnn_{idx}_hidden_memory.bin", 'rb') as p:
        hidden_memory = pickle.load(p)
    
    model.h_t1 = hidden_memory["h_t1"]
    model.c_t1 = hidden_memory["c_t1"]
    model.h_t2 = hidden_memory["h_t2"]
    model.c_t2 = hidden_memory["c_t2"]

    models.append(model)